# Langkah 1: Seleksi 1000 Review Unik

In [ ]:
import pandas as pd

# Load dataset
def load_data(file_path):
    return pd.read_csv(file_path)

# Load
data_path = "/content/drive/MyDrive/Colab Notebooks/NLP/priority_3k_labelled.csv"
data = load_data(data_path)

In [ ]:
# Lihat dataset mentah
print(data)

In [ ]:
# Seleksi 1000 review unik
def select_unique_reviews(data, n=1000):
    unique_reviews = data.drop_duplicates(subset='review', keep='first')
    return unique_reviews.sample(n=min(n, len(unique_reviews)))

selected_data = select_unique_reviews(data, 1000)

In [ ]:
# Lihat dataset sementara
print(selected_data)

# Langkah 2: Preprocessing (Cleaning, Stemming, Tokenizing)

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
!pip install Sastrawi

In [ ]:
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Membuat stemmer untuk bahasa Indonesia
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Preprocessing function untuk bahasa Indonesia
def preprocess_text(text):
    # Cleaning
    text = str(text).lower()  # Mengubah semua teks menjadi huruf kecil
    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)  # Hanya menyisakan huruf dan tanda baca ?, !, ., , ¿
    text = re.sub(r'\s+', ' ', text)  # Menghapus spasi ganda
    text = re.sub('\[.*?\]', '', text)  # Menghapus teks dalam tanda []

    # Tokenisasi dan Stemming
    tokens = word_tokenize(text)  # Memecah teks menjadi token (kata per kata)
    tokens = [stemmer.stem(token) for token in tokens if token not in stopwords.words('indonesian')]  # Stemming dan menghapus stopwords

    return ' '.join(tokens)  # Menggabungkan kembali token-token menjadi teks yang bersih

# Preprocessing
selected_data['cleaned_review'] = selected_data['review'].apply(preprocess_text)

In [ ]:
# Lihat isi dataset setelah processing
print(selected_data)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Misalkan 'selected_data' adalah DataFrame Anda
# Membuat objek LabelEncoder
label_encoder = LabelEncoder()

# Mengubah kolom 'category_sentiment' menjadi label angka
selected_data['label'] = label_encoder.fit_transform(selected_data['category_sentiment'])

# Menampilkan hasil
print(selected_data[['category_sentiment', 'label']].head())

In [ ]:
# Cek nama kolom yang tersedia di dataset
print(selected_data.columns)

# Langkah 3: Split Data

In [ ]:
from sklearn.model_selection import train_test_split

# Split Data menjadi 800 untuk Training dan 200 untuk Testing
def split_data(data, test_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(
        data['cleaned_review'], data['label'], test_size=test_size, random_state=42)
    return X_train, X_test, y_train, y_test

# Splitting data
X_train, X_test, y_train, y_test = split_data(selected_data, test_size=0.2)

In [ ]:
# Lihat data training
print(X_train)

In [ ]:
# Lihat data target training
print(y_train)

In [ ]:
print(y_train.min())

In [ ]:
print(y_train.max())

# Langkah 4: Train Data Menggunakan Naïve Bayes dan Logistic Regression

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

# A. Train Naive Bayes
def train_naive_bayes(X_train, y_train):
    model = MultinomialNB()       # Membuat instance dari model MultinomialNB
    model.fit(X_train, y_train)   # Melatih model Naive Bayes
    return model

# B. Train Logistic Regression
def train_logistic_regression(X_train, y_train):
    model = LogisticRegression(max_iter=1000) # Membuat instance dari model LogisticRegression dengan parameter max_iter=1000 untuk menentukan jumlah maksimum iterasi dalam optimasi
    model.fit(X_train, y_train)               # Melatih model regresi logistik
    return model

# Langkah 5: Vector Embedding Menggunakan TF dan TF-IDF

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# A. Vector embedding dengan TF
def vectorize_tf(X_train, X_test):
    vectorizer = CountVectorizer()  #  Menghitung frekuensi kemunculan kata dalam teks
    X_train_vec = vectorizer.fit_transform(X_train) # Menghitung dan menyimpan representasi vektor dari data train
    X_test_vec = vectorizer.transform(X_test) # menghitung dan menyimpan representasi vektor dari data test
    return X_train_vec, X_test_vec

# B. Vector embedding dengan TF-IDF
def vectorize_tfidf(X_train, X_test):
    vectorizer = TfidfVectorizer() # Menghitung bobot TF-IDF untuk setiap kata dalam teks
    X_train_vec = vectorizer.fit_transform(X_train) # Menghitung dan menyimpan representasi vektor dari data pelatihan
    X_test_vec = vectorizer.transform(X_test) # Menghasilkan representasi vektor berdasarkan model yang telah dilatih sebelumnya
    return X_train_vec, X_test_vec


# Langkah 6: Evaluasi Menggunakan Akurasi dan F1-Score

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

# Evaluate model
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test) # Memprediksi label berdasarkan data fitur pengujian
    accuracy = accuracy_score(y_test, predictions) # Menghitung akurasi prediksi dengan membandingkan label sebenarnya
    f1 = f1_score(y_test, predictions, average='weighted') # Menghitung F1 score untuk hasil prediksi dengan menggunakan metode rata-rata 'weighted', yang memperhitungkan proporsi setiap kelas
    return accuracy, f1

In [ ]:
# TF Embedding
X_train_tf, X_test_tf = vectorize_tf(X_train, X_test)  # Mengubah data latih (X_train) dan data uji (X_test) menjadi representasi embedding berbasis frekuensi kata (TF)
nb_tf = train_naive_bayes(X_train_tf, y_train)         # Melatih model Naive Bayes menggunakan data latih yang telah di-embed dengan TF
lr_tf = train_logistic_regression(X_train_tf, y_train) # Melatih model Logistic Regression menggunakan data latih yang telah di-embed dengan TF

# TF-IDF Embedding
X_train_tfidf, X_test_tfidf = vectorize_tfidf(X_train, X_test) # Menggunakan fungsi vectorize_tfidf untuk mengubah data latih (X_train) dan data uji (X_test) menjadi representasi embedding berbasis TF-IDF
nb_tfidf = train_naive_bayes(X_train_tfidf, y_train)           # Melatih model Logistic Regression menggunakan data latih yang telah di-embed dengan TF-IDF
lr_tfidf = train_logistic_regression(X_train_tfidf, y_train)   # Melatih model Logistic Regression menggunakan data latih yang telah di-embed dengan TF-IDF

# Evaluate models
nb_tf_acc, nb_tf_f1 = evaluate_model(nb_tf, X_test_tf, y_test) # Menghitung akurasi dan F1 model Niave Bayes dengan TF
lr_tf_acc, lr_tf_f1 = evaluate_model(lr_tf, X_test_tf, y_test) # Menghitung akurasi dan F1 model Logistik regression dengan TF

nb_tfidf_acc, nb_tfidf_f1 = evaluate_model(nb_tfidf, X_test_tfidf, y_test) # Menghitung akurasi dan F1 model Niave Bayes dengan TF-IDF
lr_tfidf_acc, lr_tfidf_f1 = evaluate_model(lr_tfidf, X_test_tfidf, y_test) # Menghitung akurasi dan F1 model Logistik regression dengan TF-IDF

# Print Results
print("Naive Bayes + TF: Accuracy =", nb_tf_acc, "F1 Score =", nb_tf_f1)
print("Logistic Regression + TF: Accuracy =", lr_tf_acc, "F1 Score =", lr_tf_f1)

print("Naive Bayes + TF-IDF: Accuracy =", nb_tfidf_acc, "F1 Score =", nb_tfidf_f1)
print("Logistic Regression + TF-IDF: Accuracy =", lr_tfidf_acc, "F1 Score =", lr_tfidf_f1)


# Langkah 7: Perbandingan Hasil Kedua Classifier (4 model)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data untuk visualisasi
models = ['Naive Bayes + TF', 'Logistic Regression + TF', 'Naive Bayes + TF-IDF', 'Logistic Regression + TF-IDF']
accuracies = [nb_tf_acc, lr_tf_acc, nb_tfidf_acc, lr_tfidf_acc]
f1_scores = [nb_tf_f1, lr_tf_f1, nb_tfidf_f1, lr_tfidf_f1]

x = np.arange(len(models))  # label lokasi untuk model
width = 0.35  # lebar bar

fig, ax = plt.subplots(figsize=(10, 6))

# Menambahkan bar untuk akurasi dan F1 score
bars1 = ax.bar(x - width/2, accuracies, width, label='Akurasi', color='royalblue')
bars2 = ax.bar(x + width/2, f1_scores, width, label='F1 Score', color='lightcoral')

# Menambahkan label, judul, dan legenda
ax.set_ylabel('Skor')
ax.set_title('Perbandingan Akurasi dan F1 Score dari Model')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()

# Menambahkan nilai di atas setiap bar
def add_value_labels(bars):
    """Menambahkan nilai di atas setiap bar."""
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}',  # format angka
                    xy=(bar.get_x() + bar.get_width() / 2, height),  # koordinat
                    xytext=(0, 3),  # offset
                    textcoords="offset points",
                    ha='center', va='bottom')

add_value_labels(bars1)
add_value_labels(bars2)

# Menampilkan plot
plt.tight_layout()
plt.show()

# Perbandingan hasil
1. Model Logistic Regression dengan TF adalah yang terbaik di antara kombinasi yang diuji, dengan akurasi dan F1 score tertinggi.
2. Naive Bayes, terutama dengan embedding TF-IDF, tidak menunjukkan performa yang baik, yang menunjukkan bahwa model ini mungkin tidak cocok untuk jenis data ini atau cara data diproses.
3. Embedding TF lebih efektif dibandingkan dengan TF-IDF untuk kedua model yang diuji dalam hal klasifikasi review hotel. Ini mungkin menunjukkan bahwa model lebih berhasil dalam mengidentifikasi pola berdasarkan frekuensi kemunculan kata daripada dengan memperhatikan bobot kata.

Kesimpulan: Model terbaik untuk klasifikasi review hotol berdasarkan pemrosesan di atas adalah Model Logistic Regression yang dikombinasikan dengan TF. Model tidak berlaku sama untuk semua kasus, sehingga tidak dapat disimpulkan model mana yang lebih baik.